## Testing

In [ ]:
import sys, os, json
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.scraper import get_recipe_data
from src.ingredients_parser import IngredientsParser
with open("../src/ground_truth.json", "r", encoding="utf-8") as f:
    ground_truth = json.load(f)

allrecipes_urls = [
    "https://www.allrecipes.com/recipe/21014/good-old-fashioned-pancakes/",
    "https://www.allrecipes.com/recipe/24074/alysias-basic-meat-lasagna/",
    "https://www.allrecipes.com/recipe/228293/curry-stand-chicken-tikka-masala-sauce/",
    "https://www.allrecipes.com/recipe/218091/classic-and-simple-meat-lasagna/",
    "https://www.allrecipes.com/recipe/238543/grandmas-cucumber-and-onion-salad/",
    "https://www.allrecipes.com/recipe/15375/fried-chicken-with-creamy-gravy/",
    "https://www.allrecipes.com/recipe/218288/garlic-spinach/",
    "https://www.allrecipes.com/recipe/57348/balsamic-glazed-carrots/",
    "https://www.allrecipes.com/recipe/254558/refreshing-oatmeal-drink-agua-de-avena/",
    "https://www.allrecipes.com/recipe/202975/potstickers-chinese-dumplings/"
]
epicurious_urls = [
    "https://www.epicurious.com/recipes/food/views/easy-fried-rice",
    "https://www.epicurious.com/recipes/food/views/ba-syn-easy-apple-cake",
    "https://www.epicurious.com/recipes/food/views/ba-syn-spicy-cashew-scallion-noodles",
    "https://www.epicurious.com/recipes/food/views/ba-syn-sweet-and-sour-cranberry-chicken-stir-fry",
    "https://www.epicurious.com/recipes/food/views/ba-syn-peach-blueberry-pie",
    "https://www.epicurious.com/recipes/food/views/tomato-galette",
    "https://www.epicurious.com/recipes/food/views/ba-syn-ginger-cardamom-zucchini-bread",
    "https://www.epicurious.com/recipes/food/views/white-bean-turkey-chili",
    "https://www.epicurious.com/recipes/food/views/thai-curry-puff-ga-ree-puff",
    "https://www.epicurious.com/recipes/food/views/steamed-winter-veggie-bowls"
]
bonappetit_urls = [
    "https://www.bonappetit.com/recipe/kale-pesto-with-whole-wheat-pasta",
    "https://www.bonappetit.com/recipe/adult-spaghettios",
    "https://www.bonappetit.com/recipe/ham-cheese-and-onion-empanadas",
    "https://www.bonappetit.com/recipe/crispy-smashed-potatoes-with-walnut-dressing",
    "https://www.bonappetit.com/recipe/zucchini-lentil-fritters-with-lemony-yogurt",
    "https://www.bonappetit.com/recipe/grilled-chicken-skewers-with-toum-shish-taouk",
    "https://www.bonappetit.com/recipe/bas-best-apple-pie",
    "https://www.bonappetit.com/recipe/ba-best-lasagna",
    "https://www.bonappetit.com/recipe/bas-best-chocolate-chip-cookies",
    "https://www.bonappetit.com/recipe/spicy-salmon-bowl"
]

allrecipes_data = allrecipes_urls.copy() + epicurious_urls.copy() + bonappetit_urls.copy()

In [ ]:
title, ingredients, directions = get_recipe_data(allrecipes_data[0])
ingreadient_parser = IngredientsParser(ingredients)
parsed_components = ingreadient_parser.parse()

In [ ]:
for pc in parsed_components:
    print(pc)

In [ ]:
def performance(ground_truth, parsed_components):
    ing_name_correct = 0
    ing_quantity_correct = 0
    ing_unit_correct = 0
    ing_prep_correct = 0
    ing_descriptors_correct = 0
    
    for pc in parsed_components:
        print(pc)